## Lesosn 21

[Course Repo](https://github.com/fastai/course22p2)


Breif outline

-  Jono: Demo of WandB.  (notebook 21_cifar10_and_wandb.ipynb) 
    - Uses CFAR10 instead of fashion mnist.  The tools still work. 
    - Weights and Biases (WandB) mointoring. Using [WandB](https://wandb.ai/site) for monitoring and tracking of model performance. 
    - You call logging functions in your code to log metrics, hyperparameters, and other data to the WandB server, and then you can view and compare these results in the WandB dashboard.
    - You can also use WandB to log models, code and more.
    - Jono implements a WandB callback in a few lines to implment this!
    - Jeremy says he doesn't use this (intentionally) because he worries about just doing wide sweeps of hyperparameters.  He prefers to do a more focused search.

- 20:50 Metrics  



## WandB demo

Jono: Demo of WandB.  (notebook 21_cifar10_and_wandb.ipynb) 

- Uses CFAR10 instead of fashion mnist.  The tools still work. 
- Weights and Biases (WandB) mointoring. Using [WandB](https://wandb.ai/site) for monitoring and tracking of model performance. 
- You call logging functions in your code to log metrics, hyperparameters, and other data to the WandB server, and then you can view and compare these results in the WandB dashboard.
- You can also use WandB to log models, code and more.
- Jono implements a WandB callback in a few lines to implment this!
- Jeremy says he doesn't use this (intentionally) because he worries about just doing wide sweeps of hyperparameters.  He prefers to do a more focused search.

## FID METRIC:

- Frechet inception distance (FID), demo in notbout 18_Fid.ipynb  Uses DDPM model from previous lesson. 
   - FID is a metric for generative models.
   - It is a measure of how well the generated images are similar.
   - Looks to see what the 'typical' final layer applications look like for a set up images.

At 31:00 he discovers "a bug" in the images scaline.

Discusses several ways to grab the output of the gloabl average pooling layer.

- Hooks
- Deleting the rest of the layers. 
- other ways....?

- compares means and covariance matrix of the global pooling layer (accross the samples/training data) for a classification model for the test images vs the generated images.  Correlation mans for example that images with one feature will have another feature.  So you want the generated images to have the same means and covariance matrix as the training data.

Show example for the case at hand, 512 channels. (around 38 minutes)

- The FID score is the distance between the means and covariance matrix.  
- Needs matrix squar root of hte product of the covariance matrices ..  

$$
d_{F}(\mathcal N(\mu, \Sigma), \mathcal N(\mu', \Sigma'))^2 = \lVert \mu - \mu' \rVert^2_2 + \operatorname{tr}\left(\Sigma + \Sigma' -2\left(\Sigma \Sigma'  \right)^\frac{1}{2} \right)
$$

- called Inception because it uses the Inception model in the more normal use case.

- He gets 33.8 .  Whats that supposed to tell us?

- Primary caveat is that you cant really compare FID scores between different sized data sets. (Biased somehow... )

## KID 50:00

KID Kernal inception distance.   

- Uses the features directly not the means accross the set. 
- "The math doesnt matter" 
- Measure of simularity between the distribtuions of the features.   
- Has low bias but high variance.

Jeramy creates a class for thsi that returns both FID and KID.  He shows for example how the images improve during the denoising process. KID and FID do look the same. 

Jeremy also looks at the 'real' fid using the real FID . 

## Fixing the scaling bug 1:00

- Jeremy noticed that back in the DDPM_v2 notebook, the images were scaled from 0 to 1 instad of -1 to 1.   He fixes this:
```
@inplace
def transformi(b): b[xl] = [F.pad(TF.to_tensor(o), (2,2,2,2))*2-1 for o in b[xl]]
```

- This makes everything worse, so he spent two days trying to find the other bug that must be responsble and was being offset by this bug.

- In the end he finds no relavent bug and asks: why is it a bug to scale from 0 to 1? Just because everyone else does it?  He tried it ny just subtracting 0.5 which keeps the range the same (smaller then normal). This leads to DDBM_v3. This is an improvement to the model, even accoring to the FID score.
 

### Schedule experiments 1:09

* uses 19 DDPM_v3 notebook.  

* As part of his debugging he started to question everything, for example the $\beta$ schedule

* Tested Cosine squedule vs linear schedule to compare them.  Remember that $\bar{\alpha}$ is what really matters.  

Note that the linear score has a long part of the time when the $\bar{\alpha}$ is near zero.  So he also tried decreasing $\beta_{max}$. In fact the curves are similar in that case. So in the next version he changed $\beta_{max}$ to 0.01   Results do look better. He also changed the model by making it bigger and training for longer.

Fid is nearly as good as real image. 


* SKip sampling

In [ ]:
205 % 5

In [6]:
106//100

1

In [19]:
n_steps  = 1000
[t for t in range(n_steps) if (t+101)%((t+101)//100)==0][290:]

[949, 959, 969, 979, 989, 999]

In [ ]:

@torch.no_grad()
def sample2(model, sz):
    ps = next(model.parameters())
    x_t = torch.randn(sz).to(ps)
    sample_at = {t for t in range(n_steps) if (t+101)%((t+101)//100)==0}
    preds = []
    for t in reversed(range(n_steps)):
        t_batch = torch.full((x_t.shape[0],), t, device=ps.device, dtype=torch.long)
        z = (torch.randn(x_t.shape) if t > 0 else torch.zeros(x_t.shape)).to(ps)
        ᾱ_t1 = alphabar[t-1]  if t > 0 else torch.tensor(1)
        b̄_t = 1-alphabar[t]
        b̄_t1 = 1-ᾱ_t1
        if t in sample_at: noise = model((x_t, t_batch))
        x_0_hat = ((x_t - b̄_t.sqrt() * noise)/alphabar[t].sqrt())
        x_t = x_0_hat * ᾱ_t1.sqrt()*(1-alpha[t])/b̄_t + x_t * alpha[t].sqrt()*b̄_t1/b̄_t + sigma[t]*z
        if t in sample_at: preds.append(x_t.float().cpu())
    return preds

    # And he has another that skipps even more less severly

    # FID SCORE TILL pretty good

### 1:20  DDIM

Whats the best paper for faster generation?